In [1]:
pip install -q databricks-sql-connector pandas python-dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# -*- coding: utf-8 -*-
# Databricks SQL → pandas DataFrame (Clean Code, robust, .env-Unterstützung)
from databricks import sql
import pandas as pd
import os

# -----------------------------
# 1) Konstqnzen definbierenn
# -----------------------------
DATABRICKS_SERVER_HOSTNAME = "dbc-9xxxx.cloud.databricks.com"
DATABRICKS_HTTP_PATH = "/sql/1.0/warehouses/xxxxx"
DATABRICKS_TOKEN = "dapi1xxxxxx1"

In [3]:
# -----------------------------
# Helper-Funktion: Query → DataFrame
# -----------------------------
def query_to_df(query: str) -> pd.DataFrame:
    """Führt eine SQL-Query gegen Databricks aus und gibt ein DataFrame zurück."""
    with (
        sql.connect(
            server_hostname=DATABRICKS_SERVER_HOSTNAME,
            http_path=DATABRICKS_HTTP_PATH,
            access_token=DATABRICKS_TOKEN,
        ) as conn,
        conn.cursor() as cur,
    ):
        cur.execute(query)
        rows = cur.fetchall()
        cols = [c[0] for c in cur.description]
    return pd.DataFrame(rows, columns=cols)

In [ ]:
# -----------------------------
# Beispiel 1: SQL-Query
# -----------------------------
df = query_to_df("""
    SELECT *
    FROM workspace.default.enrollments_silver
    WHERE enroll_id = 'e001'
""")

# -----------------------------
# Ergebnis anzeigen
# -----------------------------
try:
    display(df)  # Databricks / Jupyter / Colab
except NameError:
    print(df)  # Fallback für VS Code / CLI

[WARN] pyarrow is not installed by default since databricks-sql-connector 4.0.0,any arrow specific api (e.g. fetchmany_arrow) and cloud fetch will be disabled.If you need these features, please run pip install pyarrow or pip install databricks-sql-connector[pyarrow] to install


,enroll_id,student_id,email,gpa,profile,quantity,courses,processed_timestamp
0,e001,1,anna@example.com,3.9,full-time,1,[101],2025-08-01 09:00:00


In [ ]:
# -----------------------------
# Beispiel 2: Python abfragen → DataFrame
# -----------------------------
df = query_to_df(
    "SELECT * FROM workspace.default.enrollments_silver WHERE enroll_id = 'e001'"
)

# Ganz normal mit pandas weiterarbeiten
print(df.head())
print(df.shape)
print("\n")

# Spalten auswählen / filtern
print(df[["student_id", "gpa"]])

  enroll_id  student_id             email  gpa    profile  quantity courses  \
0      e001           1  anna@example.com  3.9  full-time         1   [101]   

  processed_timestamp  
0 2025-08-01 09:00:00  
(1, 8)


   student_id  gpa
0           1  3.9
